In [1]:
from IPython.display import Image, display
from ipywidgets import widgets, interact
import cantera as ct
import numpy as np

%matplotlib inline
%config InlineBackend.figure_formats = ["svg"]
from matplotlib import pyplot as plt
from collections import defaultdict
import subprocess

plt.rcParams["figure.dpi"] = 120

print(f"Using Cantera version: {ct.__version__}")

Using Cantera version: 2.6.0


In [2]:
gas = ct.Solution('newer_hcof_c2_CFG.yaml')
#gas.TPX = 1867.13, 1e5, "CH4:1.0, CF4:0.0001, O2:2.4, N2:10"
#gas.TPX = 1860, 1e5, "CH4:1, CF4:0.0001, N2:10, O2:2.4"
gas.TPX = 2400, 1e5, "H2O:0.190113878, CO2:0.095056939, N2:0.714828183, CF4:0.000001"
residence_time = 4.4
gas()


  gas:

       temperature   2400 K
          pressure   1e+05 Pa
           density   0.13848 kg/m^3
  mean mol. weight   27.634 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.4595e+05       -4.0331e+06  J
   internal energy       -8.6807e+05       -2.3988e+07  J
           entropy            9917.5        2.7405e+05  J/K
    Gibbs function       -2.3948e+07       -6.6176e+08  J
 heat capacity c_p            1526.7             42189  J/K
 heat capacity c_v            1225.8             33874  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                N2           0.72467           0.71483           -27.911
               H2O           0.12394           0.19011           -42.088
               CO2           0.15139          0.095057           -54.812
     

In [3]:
reactor = ct.IdealGasConstPressureReactor(gas, energy="on")
reactor_network = ct.ReactorNet([reactor])
reactor_network.atol = 1e-12
reactor_network.rtol = 1e-12

In [4]:
profiles = defaultdict(list)
time = 0
steps = 0
while time < residence_time:
    profiles["time"].append(time)
    profiles["pressure"].append(gas.P)
    profiles["temperature"].append(gas.T)
    profiles["mole_fractions"].append(gas.X)
    time = reactor_network.step()
    steps += 1

In [5]:
from pathlib import Path

In [6]:
@interact(
    plot_step=widgets.IntSlider(value=100, min=0, max=steps, step=10),
    threshold=widgets.FloatSlider(value=0.0, min=0, max=0.001, step=0.0001),
    details=widgets.ToggleButton(),
    species=widgets.Dropdown(
        options=gas.element_names,
        value="C",
        description="Element",
        disabled=False,
    ),
)
def plot_reaction_path_diagrams(plot_step, threshold, details, species):
    P = profiles["pressure"][plot_step]
    T = profiles["temperature"][plot_step]
    X = profiles["mole_fractions"][plot_step]
    time = profiles["time"][plot_step]
    gas.TPX = T, P, X
    print("time = {:.2g} s".format(T))

    diagram = ct.ReactionPathDiagram(gas, species)
    diagram.threshold = threshold

    diagram.show_details = details
    dot_file = "reaction_paths.dot"
    png_file = "reaction_paths.png"
    diagram.write_dot(dot_file)
    subprocess.run(f"dot {dot_file} -Tpng -o{png_file} -Gdpi=100".split())
    img = Image(filename=png_file)
    display(img)
    img_path = Path.cwd().joinpath(png_file)


interactive(children=(IntSlider(value=100, description='plot_step', max=1076, step=10), FloatSlider(value=0.0,…

In [7]:
##CH3F in this case
CH3F_stoichiometry = np.zeros_like(gas.reactions())
for i, r in enumerate(gas.reactions()):
    CH3F_moles = r.products.get("CH3F", 0) - r.reactants.get("CH3F", 0)
    CH3F_stoichiometry[i] = CH3F_moles
CH3F_reaction_indices = CH3F_stoichiometry.nonzero()[0]

In [8]:
profiles["CH3F_production_rates"] = []
for i in range(len(profiles["time"])):
    X = profiles["mole_fractions"][i]
    t = profiles["time"][i]
    T = profiles["temperature"][i]
    P = profiles["pressure"][i]
    gas.TPX = (T, P, X)
    CH3F_production_rates = (
        gas.net_rates_of_progress
        * CH3F_stoichiometry  #  [kmol/m^3/s]
        * gas.volume_mass  # Specific volume [m^3/kg].
    )  # overall, mol/s/g  (g total in reactor, same basis as N_atoms_in_fuel)

    profiles["CH3F_production_rates"].append(
        CH3F_production_rates[CH3F_reaction_indices]
    )

In [9]:
@interact(
    annotation_cutoff=widgets.FloatSlider(value=1e-10, min=1e-10, max=4, steps=10),
    profiles=widgets.fixed(profiles),
)
def plot_instantaneous_fluxes(profiles, annotation_cutoff):
    profiles = profiles
    fig = plt.figure(figsize=(6, 6))
    plt.plot(profiles["time"], np.array(profiles["CH3F_production_rates"]))

    for i, CH3F_production_rate in enumerate(
        np.array(profiles["CH3F_production_rates"]).T
    ):
        peak_index = abs(CH3F_production_rate).argmax()
        peak_time = profiles["time"][peak_index]
        peak_CH3F_production = CH3F_production_rate[peak_index]
        reaction_string = gas.reaction_equations(CH3F_reaction_indices)[i]

        if abs(peak_CH3F_production) > annotation_cutoff:
            plt.annotate(
                (reaction_string).replace("2", "$_2$").replace("<=>", "="),
                xy=(peak_time, peak_CH3F_production),
                xytext=(
                    peak_time * 2,
                    (
                        peak_CH3F_production
                        + 0.000003
                        * (peak_CH3F_production / abs(peak_CH3F_production))
                        * (abs(peak_CH3F_production) > 0.000005)
                        * (peak_CH3F_production < 0.00006)
                    ),
                ),
                arrowprops=dict(
                    arrowstyle="->",
                    color="black",
                    relpos=(0, 0.6),
                    linewidth=2,
                ),
                horizontalalignment="left",
            )

    plt.xlabel("Time (s)", fontsize=16)
    plt.ylabel("Net rates of CH3F production", fontsize=16)
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=1e-10, description='annotation_cutoff', max=4.0, min=1e-10), Output())…